## Scholix

### Pass the dataset DOIs to the scholex API to get the citations and their respective DOIs

In [1]:
import requests, time, json, re, datetime, os, sys
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import numpy as np
import pandas as pd
from math import ceil
from datetime import date
# from scholix_fun import getNERCDataDOIs, getScholixDatasetCitations, process_citation_results, getPublicationType, countScholixCitations, getCitationString

# import exportCitationResultsToCsv
# from Results import convertCSVtoJSON


# sys.path.insert(0, '..')

from citations_fun.getScholixCitations import getScholixCitations
from citations_fun.processScholixCitations import process_citation_results


In [ ]:
with open("results/nerc_datacite_dois.json") as f:
    nerc_datacite_dois = json.load(f)

nerc_datacite_dois_df = pd.DataFrame(nerc_datacite_dois)



In [ ]:
nerc_datacite_dois_df.head()

In [ ]:
# this takes about 23 - 28 mins

scholex_df = getScholixCitations(nerc_datacite_dois_df)

In [ ]:
scholex_df.head()


In [4]:
scholex_df.columns

Index(['relationshipType', 'pub_title', 'pub_date', 'pub_authors', 'pub_doi',
       'data_doi', 'data_publisher', 'data_title', 'data_authors'],
      dtype='object')

#### Process the citation results

In [5]:
scholex_df_processed = process_citation_results(scholex_df)

In [6]:
scholex_df_processed.columns

Index(['relationshipType', 'pub_title', 'pub_date', 'pub_doi', 'data_doi',
       'data_title', 'data_authors', 'data_publisher', 'pub_authors'],
      dtype='object')

### output latest_results_scholex.csv

In [7]:
scholex_df_processed.to_csv("results/latest_results_scholex.csv", index= False )

### can move this to the get publication info script

### Check the DOIs at DOI.org to determine the type of publication and to check there are no duplicates (by preprints etc) 

In [ ]:
# very long 3+ hours
scholex_df = getPublicationType.getPublicationType(scholex_df)

### Output spreadsheet

In [ ]:
today = date.today()

scholex_filename = "Results/Intermediate data/" + 'scholix_citation_publication_info_' + (today.strftime("%d%m%Y")) + '.csv'
scholex_df.to_csv(scholex_filename, index = False)
scholex_df.to_csv("Results/Intermediate data/latest_results_scholix.csv", index = False)

In [ ]:
import pickle
import types

def get_pickleable_globals():
    safe_vars = {}
    for name, val in globals().items():
        if name.startswith("__") or isinstance(val, (
            types.ModuleType,
            types.FunctionType,
            types.BuiltinFunctionType,
            types.MethodType,
            types.BuiltinMethodType,
            types.CodeType,
            types.TracebackType,
            types.FrameType,
        )):
            continue
        try:
            pickle.dumps(val)
            safe_vars[name] = val
        except Exception:
            continue
    return safe_vars

# Save workspace
workspace = get_pickleable_globals()

# Check what you’re saving
print(f"Saving {len(workspace)} variables:", list(workspace.keys()))

# Only save if it's not empty
if workspace:
    with open('workspace.pkl', 'wb') as f:
        pickle.dump(workspace, f)
else:
    print("No variables to save.")


In [2]:
import pickle
try:
    with open('workspace.pkl', 'rb') as f:
        loaded_vars = pickle.load(f)
        globals().update(loaded_vars)
        print(f"Loaded {len(loaded_vars)} variables.")
except EOFError:
    print("File is empty. Nothing to load.")
except FileNotFoundError:
    print("workspace.pkl not found.")


Loaded 26 variables.


In [ ]:
import os
os.path.getsize('workspace.pkl')  # Should be more than 0
